# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## 0. Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it and add it to you `.gitignore` file. 

You will find more info about virtual environments in the course content and on the web.

## 1. API Scraping

### 1a. A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword. NOTE: external libraries need to be installed first. Check the [request Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the documentation to:

1. Use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. Check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is an explanation of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [145]:
# import the requests library (1 line)
import requests

# (optional) we'll use pandas later for CSV/JSON handling
import pandas

# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
root_url = "https://country-leaders.onrender.com"

# assign the /status endpoint to another variable called status_url (1 line)
status_url = f"{root_url}/status"

# query the /status endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(status_url)

# check the status_code using a condition and print appropriate messages (4 lines)
if req.status_code == 200:
    print(req.text)
else:
    print(req.status_code)


"Alive"


### 1b. Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. 


In [146]:
# Set the countries_url variable (1 line)
countries_url = f"{root_url}/countries"

# query the /countries endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(countries_url)

# Get the JSON content and store it in the countries variable (1 line)
countries = req.json()

# display the request's status code and the countries variable (1 line)
print(req.status_code, countries)


403 {'message': 'The cookie is missing'}


### 1c. Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [147]:
# Set the cookie_url variable (1 line)
cookie_url = f"{root_url}/cookie"

# Query the endpoint, set the cookies variable and display it (2 lines)
response = requests.get(cookie_url)
cookies = response.cookies
cookies


<RequestsCookieJar[Cookie(version=0, name='user_cookie', value='364db5ea-eb47-421b-aacb-a427540bedca', port=None, port_specified=False, domain='country-leaders.onrender.com', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'SameSite': 'lax'}, rfc2109=False)]>

Try to query the countries endpoint using the cookie, save the output and print it.

In [148]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
countries = requests.get(countries_url, cookies=cookies).json()

# display the countries variable (1 line)
print(countries)


['be', 'us', 'ru', 'fr', 'ma']


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### 1d. Getting the actual data from the API

Query the `/leaders` endpoint.

In [149]:
# Set the leaders_url variable (1 line)
leaders_url = f"{root_url}/leaders"

# Choose one country and store it in the country variable (1 line)
country = countries[0]

# Query the /leaders endpoint for the chosen country using the cookie (2 lines)
response = requests.get(f"{leaders_url}?country={country}", cookies=cookies)
leaders = response.json()

# Display the country's name and the leaders variable (2 lines)
print(country)
print(leaders)


be
[{'id': 'Q12978', 'first_name': 'Guy', 'last_name': 'Verhofstadt', 'birth_date': '1953-04-11', 'death_date': None, 'place_of_birth': 'Dendermonde', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'start_mandate': '1999-07-12', 'end_mandate': '2008-03-20'}, {'id': 'Q12981', 'first_name': 'Yves', 'last_name': 'Leterme', 'birth_date': '1960-10-06', 'death_date': None, 'place_of_birth': 'Wervik', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme', 'start_mandate': '2009-11-25', 'end_mandate': '2011-12-06'}, {'id': 'Q12983', 'first_name': 'Herman', 'last_name': 'None', 'birth_date': '1947-10-31', 'death_date': None, 'place_of_birth': 'Etterbeek', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy', 'start_mandate': '2008-12-30', 'end_mandate': '2009-11-25'}, {'id': 'Q14989', 'first_name': 'Léon', 'last_name': 'Delacroix', 'birth_date': '1867-12-27', 'death_date': '1929-10-15', 'place_of_birth': 'Saint-Josse-ten-Noode', 'wikipedia_url': 'https:/

It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [150]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)
# assign the output to the leaders variable (1 line)
leaders = requests.get(leaders_url, cookies=cookies, params={"country": countries[4]}).json()

# display the leaders variable (1 line)
print(leaders)


[{'id': 'Q57553', 'first_name': 'Mohammed', 'last_name': 'None', 'birth_date': '1963-08-21', 'death_date': None, 'place_of_birth': 'Rabat', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86', 'start_mandate': '1999-07-23', 'end_mandate': None}, {'id': 'Q69103', 'first_name': 'Hassan', 'last_name': 'None', 'birth_date': '1929-07-09', 'death_date': '1999-07-20', 'place_of_birth': 'Rabat', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF', 'start_mandate': '1961-02-26', 'end_mandate': '1999-07-23'}, {'id': 'Q193874', 'first_name': 'Mohammed', 'last_name': 'None', 'birth_date': '1909-08-10', 'death_date': '1961-02-26', 'place_of_birth': 'Fez', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%AE%D8%A7%D9%85%D8%B3_%D8%A8%D9%86_%D9%8A%D9%88%D8%B3%D

### 1e. A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [151]:
# 4 lines
# Initialize the leaders_per_country dictionary (1 line)
leaders_per_country = {}

# Loop through countries, query the /leaders endpoint with cookies and parameters, and store results (2 lines)
for country in countries:
    leaders_per_country[country] = requests.get(leaders_url, cookies=cookies, params={"country": country}).json()

# Display one sample to check the structure (1 line)
print(countries[0], leaders_per_country[countries[0]][0])


be {'id': 'Q12978', 'first_name': 'Guy', 'last_name': 'Verhofstadt', 'birth_date': '1953-04-11', 'death_date': None, 'place_of_birth': 'Dendermonde', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'start_mandate': '1999-07-12', 'end_mandate': '2008-03-20'}


In [152]:
# Create a dictionary of all leaders per country in one line (1 line)
leaders_per_country = {country: requests.get(leaders_url, cookies=cookies, params={"country": country}).json() for country in countries}

#Extra print 
print(leaders_per_country)

{'be': [{'id': 'Q12978', 'first_name': 'Guy', 'last_name': 'Verhofstadt', 'birth_date': '1953-04-11', 'death_date': None, 'place_of_birth': 'Dendermonde', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'start_mandate': '1999-07-12', 'end_mandate': '2008-03-20'}, {'id': 'Q12981', 'first_name': 'Yves', 'last_name': 'Leterme', 'birth_date': '1960-10-06', 'death_date': None, 'place_of_birth': 'Wervik', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme', 'start_mandate': '2009-11-25', 'end_mandate': '2011-12-06'}, {'id': 'Q12983', 'first_name': 'Herman', 'last_name': 'None', 'birth_date': '1947-10-31', 'death_date': None, 'place_of_birth': 'Etterbeek', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy', 'start_mandate': '2008-12-30', 'end_mandate': '2009-11-25'}, {'id': 'Q14989', 'first_name': 'Léon', 'last_name': 'Delacroix', 'birth_date': '1867-12-27', 'death_date': '1929-10-15', 'place_of_birth': 'Saint-Josse-ten-Noode', 'wikipedia_url': 'htt

It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [153]:
# < 15 lines
# Define the get_leaders() function (11 lines)
def get_leaders():
    root_url = "https://country-leaders.onrender.com"
    cookie_url = f"{root_url}/cookie"
    countries_url = f"{root_url}/countries"
    leaders_url = f"{root_url}/leaders"

    cookies = requests.get(cookie_url).cookies
    countries = requests.get(countries_url, cookies=cookies).json()

    leaders_per_country = {}
    for country in countries:
        leaders_per_country[country] = requests.get(leaders_url, cookies=cookies, params={"country": country}).json()

    return leaders_per_country


Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [154]:
# 2 lines
# Test the function and store the result (1 line)
leaders_per_country = get_leaders()

# Display the keys (countries) and the number of leaders for a quick check (1 line)
print(leaders_per_country)


{'be': [{'id': 'Q12978', 'first_name': 'Guy', 'last_name': 'Verhofstadt', 'birth_date': '1953-04-11', 'death_date': None, 'place_of_birth': 'Dendermonde', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'start_mandate': '1999-07-12', 'end_mandate': '2008-03-20'}, {'id': 'Q12981', 'first_name': 'Yves', 'last_name': 'Leterme', 'birth_date': '1960-10-06', 'death_date': None, 'place_of_birth': 'Wervik', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme', 'start_mandate': '2009-11-25', 'end_mandate': '2011-12-06'}, {'id': 'Q12983', 'first_name': 'Herman', 'last_name': 'None', 'birth_date': '1947-10-31', 'death_date': None, 'place_of_birth': 'Etterbeek', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy', 'start_mandate': '2008-12-30', 'end_mandate': '2009-11-25'}, {'id': 'Q14989', 'first_name': 'Léon', 'last_name': 'Delacroix', 'birth_date': '1867-12-27', 'death_date': '1929-10-15', 'place_of_birth': 'Saint-Josse-ten-Noode', 'wikipedia_url': 'htt

## 2. Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [66]:
# 3 lines

""" # Select one leader's Wikipedia URL and assign it to the wiki_url variable (1 line)
wiki_url = leaders_per_country[countries[0]][0]["wikipedia_url"]

# Query the Wikipedia page and store the response (1 line)
response = requests.get(wiki_url)

# Display the text content of the page (1 line)
print(response.text)
 """

# Select one leader's Wikipedia URL and assign it to the wiki_url variable (1 line)
wiki_url = leaders_per_country["us"][0]["wikipedia_url"]

# Query the Wikipedia page with a proper User-Agent header (1 line)
response = requests.get(wiki_url, headers={"User-Agent": "Mozilla/5.0 (Wikipedia Scraper Project - Educational Use)"})

# Display the text content of the page (1 line)
print(response.text[:1000])  # Display first 1000 chars to avoid flooding the notebook




<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" lang="en" dir="ltr">
<head>
<meta charset="UTF-8">
<title>George Washington - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-f

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [102]:
# 3 lines

# Import the BeautifulSoup class from bs4 (1 line)
from bs4 import BeautifulSoup

# Load the HTML content using BeautifulSoup (1 line)
soup = BeautifulSoup(response.text, "html.parser")

# Use prettify() to visualize the structure (1 line)
print(soup.prettify()[:1000])  # Limit output to 1000 chars for readability



<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" dir="rtl" lang="ar">
 <head>
  <meta charset="utf-8"/>
  <title>
   الحسن الثاني بن محمد - ويكيبيديا
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-co

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [101]:
# 2 lines
# Extract all HTML paragraphs and store them in the paragraphs variable (1 line)
paragraphs = soup.find_all("p")

# Display how many paragraphs were found (1 line)
print(f"Found {len(paragraphs)} paragraphs")

# Show the first few paragraphs to inspect (1 line)
print(paragraphs[:3])


Found 150 paragraphs
[<p><b>الحَسَن الثاني بِنْ مُحمد بِنْ يوسف العَلوي</b> (9 يوليو 1929 – 23 يوليو 1999)<sup class="reference" id="cite_ref-archive.org2_13-0"><a href="#cite_note-archive.org2-13"><span class="cite-bracket">[</span>13<span class="cite-bracket">]</span></a></sup> هو ثاني ملوك <a href="/wiki/%D8%A7%D9%84%D9%85%D8%BA%D8%B1%D8%A8" title="المغرب">المملكة المغربية</a> بعد الإستقلال، والملك الثاني والعشرون للمغرب من <a href="/wiki/%D8%B3%D9%84%D8%A7%D9%84%D8%A9_%D8%A7%D9%84%D8%B9%D9%84%D9%88%D9%8A%D9%8A%D9%86_%D8%A7%D9%84%D9%81%D9%8A%D9%84%D8%A7%D9%84%D9%8A%D9%8A%D9%86_%D8%A7%D9%84%D8%AD%D8%A7%D9%83%D9%85%D8%A9" title="سلالة العلويين الفيلاليين الحاكمة">سلالة العلويين الفيلاليين</a>، تولى حكم المملكة المغربية خلفًا لوالده الملك <a href="/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%AE%D8%A7%D9%85%D8%B3_%D8%A8%D9%86_%D9%8A%D9%88%D8%B3%D9%81" title="محمد الخامس بن يوسف">محمد الخامس</a> في 26 فبراير 1961 وحتى وفاته في 23 يوليو 1999. ينتمي الملك الحسن الثاني إلى السلالة العلوية

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one:
   * Store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
   * Exit the loop

In [100]:
# 1) Loop over the HTML paragraphs
paragraphs = soup.find_all("p")

first_paragraph = None
for p in paragraphs:
    text = p.get_text(strip=True)
    # Heuristic: skip empty or very short paragraphs (nav/meta/fillers)
    if len(text) > 60:
        # 2) Identified the correct one
        first_paragraph = text
        #    * Store in first_paragraph
        #    * Exit the loop
        break

# Optional: sanity check
if first_paragraph is None:
    print("No meaningful paragraph found.")
else:
    print(first_paragraph)


الحَسَن الثاني بِنْ مُحمد بِنْ يوسف العَلوي(9 يوليو 1929 – 23 يوليو 1999)[13]هو ثاني ملوكالمملكة المغربيةبعد الإستقلال، والملك الثاني والعشرون للمغرب منسلالة العلويين الفيلاليين، تولى حكم المملكة المغربية خلفًا لوالده الملكمحمد الخامسفي 26 فبراير 1961 وحتى وفاته في 23 يوليو 1999. ينتمي الملك الحسن الثاني إلى السلالة العلوية التي تعود في نسبها إلىالحسن بن علي بن أبي طالب، وتحكم المغرب منذ عام 1666 ميلادية ويلقب الحاكم منهم بأمير المؤمنين.


At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [110]:
def get_first_paragraph(url):
    """Fetch a Wikipedia page and return the first meaningful intro paragraph."""
    import requests
    from bs4 import BeautifulSoup

    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(response.text, "html.parser")

    first_paragraph = None
    for p in soup.find_all("p"):
        text = p.get_text(" ", strip=True)  # 👈 add a space between tags
        if len(text) > 60:
            first_paragraph = text
            break

    return first_paragraph


In [111]:
# Test: 3 lines
test_url = leaders_per_country["us"][0]["wikipedia_url"]
print(get_first_paragraph(test_url))


George Washington (February 22, 1732 [ O.S. February 11, 1731] [ a ] – December 14 , 1799) was a Founding Father and the first president of the United States , serving from 1789 to 1797. As commander of the Continental Army , Washington led Patriot forces to victory in the American Revolutionary War against the British Empire . He is commonly known as the Father of the Nation for his role in bringing about American independence .


### 2a. Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [116]:
test_url = leaders_per_country["ma"][2]["wikipedia_url"]
print(get_first_paragraph(test_url))

محمد الخامس بن يوسف بن الحسن بن محمد بن عبد الرحمن بن هشام بن محمد بن عبد الله بن إسماعيل بن الشريف بن علي العلوي وُلد ( 1327 هـ / 10 أغسطس 1909م بالقصر السلطاني بفاس ) [ 1 ] وتوفي ( 1381 هـ / 26 فبراير 1961م بالرباط ) خَلَف والده السلطان مولاي يوسف الذي توفي بُكرة يوم الخميس 22 جمادى الأولى سنة 1346 هـ موافق 17 نوفمبر سنة 1927م [ 2 ] فبويع ابنه سيدي محمد سلطانا للمغرب في اليوم الموالي بعد صلاة الجمعة 23 جمادى الأولى سنة 1346 هـ موافق 18 نوفمبر سنة 1927م في القصر السلطاني بفاس [ 3 ] ولم يزل سلطان المغرب إلى سنة 1957م ، قضى منها المنفى بين ( 1953 - 1955 )، ثم اتخذ لقب الملك سنة 1957م ولم يزل ملكا إلى وفاته سنة 1961م ، ساند السلطان محمد الخامس نضالات الحركة الوطنية المغربية المطالبة بتحقيق الاستقلال ، الشيء الذي دفعه إلى الاصطدام بسلطات الحماية . وكانت النتيجة قيام سلطات الحماية بنفيه إلى مدغشقر . وعلى إثر ذلك اندلعت مظاهرات مطالبة بعودته إلى وطنه. وأمام اشتداد حدة المظاهرات، قبلت السلطات الفرنسية بإرجاع السلطان إلى عرشه يوم 16 نوفمبر 1955 . وبعد بضعة شهور تم إعلان استقلال المغرب . كان ا

In [117]:
# 3 lines

# import the regex module (1 line)
import re

# clean the paragraph by removing reference markers and brackets (1 line)
clean_paragraph = re.sub(r'\[[^\]]*\]', '', get_first_paragraph(test_url))

# display the cleaned text (1 line)
print(clean_paragraph.strip())


محمد الخامس بن يوسف بن الحسن بن محمد بن عبد الرحمن بن هشام بن محمد بن عبد الله بن إسماعيل بن الشريف بن علي العلوي وُلد ( 1327 هـ / 10 أغسطس 1909م بالقصر السلطاني بفاس )  وتوفي ( 1381 هـ / 26 فبراير 1961م بالرباط ) خَلَف والده السلطان مولاي يوسف الذي توفي بُكرة يوم الخميس 22 جمادى الأولى سنة 1346 هـ موافق 17 نوفمبر سنة 1927م  فبويع ابنه سيدي محمد سلطانا للمغرب في اليوم الموالي بعد صلاة الجمعة 23 جمادى الأولى سنة 1346 هـ موافق 18 نوفمبر سنة 1927م في القصر السلطاني بفاس  ولم يزل سلطان المغرب إلى سنة 1957م ، قضى منها المنفى بين ( 1953 - 1955 )، ثم اتخذ لقب الملك سنة 1957م ولم يزل ملكا إلى وفاته سنة 1961م ، ساند السلطان محمد الخامس نضالات الحركة الوطنية المغربية المطالبة بتحقيق الاستقلال ، الشيء الذي دفعه إلى الاصطدام بسلطات الحماية . وكانت النتيجة قيام سلطات الحماية بنفيه إلى مدغشقر . وعلى إثر ذلك اندلعت مظاهرات مطالبة بعودته إلى وطنه. وأمام اشتداد حدة المظاهرات، قبلت السلطات الفرنسية بإرجاع السلطان إلى عرشه يوم 16 نوفمبر 1955 . وبعد بضعة شهور تم إعلان استقلال المغرب . كان الملك محمد الخام

Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [119]:
# 10 lines
import requests
from bs4 import BeautifulSoup
import re

def get_first_paragraph(url):
    """Fetch a Wikipedia page, extract and clean the first meaningful intro paragraph."""
    
    # query the Wikipedia URL
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    
    # parse the HTML using BeautifulSoup
    soup = BeautifulSoup(response.text, "html.parser")
    
    # initialize variable
    first_paragraph = None
    
    # loop over HTML paragraphs
    for p in soup.find_all("p"):
        text = p.get_text(" ", strip=True)
        if len(text) > 60:
            first_paragraph = text
            break

    if not first_paragraph:
        return None

    # --- Apply regex cleaning before returning ---
    # remove reference markers like [1], [a], etc.
    cleaned = re.sub(r'\[[^\]]*\]', '', first_paragraph)
    # remove phonetic pronunciations like /ˈwɒʃɪŋtən/
    cleaned = re.sub(r'/[^/]+/', '', cleaned)
    # collapse extra spaces
    cleaned = re.sub(r'\s{2,}', ' ', cleaned).strip()
    
    return cleaned


In [121]:
test_url = leaders_per_country["us"][2]["wikipedia_url"]
print(get_first_paragraph(test_url))

Abraham Lincoln (February 12, 1809 – April 15, 1865) was the 16th president of the United States , serving from 1861 until his assassination in 1865. He led the United States through the American Civil War , defeating the Confederate States and playing a major role in the abolition of slavery .


## 3. Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [131]:
# < 20 lines

def get_leaders():
    """Retrieve leaders for all countries and enrich them with Wikipedia intro paragraphs."""
    import requests

    root_url = "https://country-leaders.onrender.com"
    cookie_url = f"{root_url}/cookie"
    countries_url = f"{root_url}/countries"
    leaders_url = f"{root_url}/leaders"

    # Get cookie (1 line)
    cookies = requests.get(cookie_url).cookies

    # Get countries (1 line)
    countries = requests.get(countries_url, cookies=cookies).json()

    # Initialize container (1 line)
    leaders_per_country = {}

    # Loop through each country (3 lines)
    for country in countries:
        leaders = requests.get(leaders_url, cookies=cookies, params={"country": country}).json()

        # Inner loop: scrape first paragraph for each leader (4 lines)
        for leader in leaders:
            wiki_url = leader.get("wikipedia_url")
            leader["first_paragraph"] = get_first_paragraph(wiki_url) if wiki_url else None

        # Save to main dictionary (1 line)
        leaders_per_country[country] = leaders

    # Return results (1 line)
    return leaders_per_country


In [132]:
# Check the output of your function (2 lines)
leaders_per_country = get_leaders()
print(leaders_per_country["us"][0])



KeyboardInterrupt: 

Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [ ]:
# < 25 lines

def get_leaders():
    """Retrieve leaders for all countries and enrich them with Wikipedia intro paragraphs."""
    import requests
    import time

    root_url = "https://country-leaders.onrender.com"
    cookie_url = f"{root_url}/cookie"
    countries_url = f"{root_url}/countries"
    leaders_url = f"{root_url}/leaders"

    # Function to get a fresh cookie
    def get_cookie():
        return requests.get(cookie_url).cookies

    # Initialize cookies and data container
    cookies = get_cookie()
    leaders_per_country = {}

    try:
        # Get list of countries
        countries = requests.get(countries_url, cookies=cookies).json()

        for country in countries:
            try:
                # Query leaders API
                resp = requests.get(leaders_url, cookies=cookies, params={"country": country})

                # Detect expired cookie or API error
                if resp.status_code != 200 or "cookie" in resp.text.lower():
                    print(f"Cookie expired while fetching {country}. Refreshing...")
                    cookies = get_cookie()
                    resp = requests.get(leaders_url, cookies=cookies, params={"country": country})

                leaders = resp.json()

                # Skip invalid responses
                if not isinstance(leaders, list):
                    print(f"Skipping {country}: invalid API response")
                    continue

                # Loop over each leader to enrich with Wikipedia intro
                for leader in leaders:
                    wiki_url = leader.get("wikipedia_url")
                    leader["first_paragraph"] = get_first_paragraph(wiki_url) if wiki_url else None
                    time.sleep(0.5)  # polite delay to avoid request throttling

                # Store the country's data
                leaders_per_country[country] = leaders

            except Exception as e:
                print(f"Error processing {country}: {e}")
                continue

    except Exception as e:
        print(f"General error: {e}")

    return leaders_per_country


Check the output of your function again.

In [ ]:
# Check the output of your function (2 lines)
leaders_per_country = get_leaders()
print(leaders_per_country["us"][0])

🔁 Cookie expired while fetching us. Refreshing...
🔁 Cookie expired while fetching ru. Refreshing...
🔁 Cookie expired while fetching ma. Refreshing...
{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04', 'first_paragraph': 'George Washington (February 22, 1732 – December 14 , 1799) was a Founding Father and the first president of the United States , serving from 1789 to 1797. As commander of the Continental Army , Washington led Patriot forces to victory in the American Revolutionary War against the British Empire . He is commonly known as the Father of the Nation for his role in bringing about American independence .'}


Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [ ]:
# < 20 lines
def get_first_paragraph(url, session=None):
    """Fetch and clean the first paragraph of a Wikipedia page using a shared session."""
    
    # use provided session or create a temporary one
    session = session or requests.Session()
    
    # query the Wikipedia URL (using session)
    response = session.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    
    # parse HTML
    soup = BeautifulSoup(response.text, "html.parser")
    
    # extract the first meaningful paragraph
    first_paragraph = None
    for p in soup.find_all("p"):
        text = p.get_text(" ", strip=True)
        if len(text) > 60:
            first_paragraph = text
            break
    
    if not first_paragraph:
        return None

    # clean the text with regex
    cleaned = re.sub(r'\[[^\]]*\]', '', first_paragraph)   # remove [1], [a], etc.
    cleaned = re.sub(r'/[^/]+/', '', cleaned)              # remove phonetic /ˈ.../
    cleaned = re.sub(r'\s{2,}', ' ', cleaned).strip()      # collapse spaces

    return cleaned


In [162]:
with requests.Session() as s:
    test_url = "https://en.wikipedia.org/wiki/George_Washington"
    print(get_first_paragraph(test_url, s))


George Washington (February 22, 1732 – December 14 , 1799) was a Founding Father and the first president of the United States , serving from 1789 to 1797. As commander of the Continental Army , Washington led Patriot forces to victory in the American Revolutionary War against the British Empire . He is commonly known as the Father of the Nation for his role in bringing about American independence .


Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [163]:
import requests

def get_leaders():
    root_url = "https://country-leaders.onrender.com"
    cookie_url = f"{root_url}/cookie"
    countries_url = f"{root_url}/countries"
    leaders_url = f"{root_url}/leaders"

    # Existing API calls (unchanged)
    cookies = requests.get(cookie_url).cookies
    countries = requests.get(countries_url, cookies=cookies).json()

    leaders_per_country = {}
    for country in countries:
        leaders_per_country[country] = requests.get(
            leaders_url, cookies=cookies, params={"country": country}
        ).json()

    # Create a single session for ALL Wikipedia calls and return it
    wiki_session = requests.Session()
    return leaders_per_country, wiki_session


In [166]:
# Call get_leaders() — this now returns both the leaders data and the wiki session
leaders_per_country, wiki_session = get_leaders()

# Select the first Moroccan leader for example (country code "ma")
leader = leaders_per_country["us"][0]
url = leader.get("wikipedia_url")

# Get the first paragraph using the same session
leader["first_paragraph"] = get_first_paragraph(url, session=wiki_session) if url else None

# Print the results
print(f"Leader name: {leader['first_name']} {leader['last_name']}")
print(f"Wikipedia URL: {url}")
print("\nExtracted first paragraph:\n")
print(leader["first_paragraph"])


Leader name: George Washington
Wikipedia URL: https://en.wikipedia.org/wiki/George_Washington

Extracted first paragraph:

George Washington (February 22, 1732 – December 14 , 1799) was a Founding Father and the first president of the United States , serving from 1789 to 1797. As commander of the Continental Army , Washington led Patriot forces to victory in the American Revolutionary War against the British Empire . He is commonly known as the Father of the Nation for his role in bringing about American independence .


## 4. Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [26]:
# 3 lines


Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [27]:
# 3 lines


Make a function `save(leaders_per_country)` to call this code easily.

In [28]:
# 3 lines


In [29]:
# Call the function (1 line)


## 5. Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## (Optional) To go further

If you want to practice scraping, you can read this section and tackle the exercises.

1. Restructure your code by using OOP (see ReadMe).
2. You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!